In [ ]:
from _install import install
await install()

## API client

`api.API` is a thin wrapper around the [HERE search & geocoding API](https://www.here.com/docs/bundle/geocoding-and-search-api-v7-api-reference/page/index.html). 
API objects host the `api_key` variable (prompted if not provided as argument `API_KEY` environment variable or in a `demo-config.json` file).

In [ ]:
from here_search.demo.api import API, HTTPSession

api = API()
session = HTTPSession()

The API class uses async methods to take advantage of Jupyter event loop and [async REPL](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7). For example `/discover` requests are sent with `api.discover()` like:

In [ ]:
resp = await api.discover(q="berlin", latitude=52, longitude=13, session=session)
resp.req.full

In [ ]:
resp.data

In [ ]:
resp = await api.autosuggest(q="restaura", latitude=52, longitude=13, limit=2, termsLimit=3, session=session)
resp.req.full

In [ ]:
resp.data

In [ ]:
resp.terms

In [ ]:
resp.geojson()

## Suggestions for a growing query

Autosuggest is made to help end-user to quickly formulate a query.


It is recommended to use Autosuggest response items `resulType` field to decide what to do in case of user selection. The selection of a `chainQuery` or `categoryQuery` result should lead to a GET of the `href` field value. Other selections should lead to a call to `/lookup` using the `id` field value. 
Note that the `resulType` field can be used to render results differently. For example, a <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Font_Awesome_5_solid_search.svg" style="width:12px"/> can be used to signal a `chainQuery` or `categoryQuery`.

The following snippet sends an Autosuggest request for each additional character of the query "restaurant hamburg" a hypothetic user intends to type to get restaurants near the German city of Hamburg. Only each response **first result** is displayed: 

In [ ]:
from functools import partial

q = "restaurant hamburg"
autosuggest = partial(api.autosuggest, latitude=52, longitude=13,  lang="en", limit=1, session=session)

template = "{:<2} {:<25} {:<18} {:<65} {:<14}"
print(template.format("#", "query", "resultType", "id", "title"))
for i in range(len(q)):
    resp = await autosuggest(q=q[: i + 1])
    item = resp.data["items"][0]
    print(template.format(i + 1, f"'{q[: i + 1]}'", item["resultType"], item["id"][:60], item["title"]))

If the end-user was actually typing the query "restaurant", the suggestion of `id` value `here:cm:taxonomy:restaurant` returned for the 4-letter query "rest" could be selected to trigger the related `href` query. But the end-user needs to continue typing untill "restaurants ham" to get the suggestion titled "Restaurants near Hamburg, Germany".

## Terms suggestions

To further help end-user to formulate such a long query, Autosuggest also returns predictive text for the query last token while it is being typed. Those suggested terms are in the response `queryTerms` when the request contains a positive `termsLimit` parameter. In the snippet below, 3 last token suggestiosn are returned per response:

In [ ]:
autosuggest = partial(autosuggest, termsLimit=3)

template = "{:<2} {:<25} {:<38} {:<14}"
print(template.format("#", "query", "terms", "title"))
for i in range(len(q)):
    resp = await autosuggest(q=q[: i + 1])
    item = resp.data["items"][0]
    terms = ",".join(t["term"] for t in resp.data["queryTerms"])
    print(template.format(i + 1, f"'{q[: i + 1]}'", terms, item["title"]))

When presented to the end-user to replace the last token in the query, `queryTerms` can significantly speed-up the query submission. In the previous interaction, the use of the token "Restaurant" returned for the "res" query, would have led to the series of 8 queries instead of 14: "r", "re", "res", "restaurant", "restaurant ", "restaurant h", "restaurant ha", "restaurant hamburg".

In [ ]:
template = "{:<2} {:<25} {:<38} {:<14}"
print(template.format("#", "query", "terms", "title"))
qs = ["r", "re", "res", "restaurant", "restaurant ", "restaurant h", "restaurant ha", "restaurant hamburg"]
for i, q in enumerate(qs):
    resp = await autosuggest(q=q)
    item = resp.data["items"][0]
    terms = ",".join(t["term"] for t in resp.data["queryTerms"])
    print(template.format(i + 1, f"'{q}'", terms, item["title"]))

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>